Here is the meat and potatoes of actually determining the TRD loci. Will still need a bit of refinement, but the idea is to look at opposite homozygous loci (of the parent strains), filter out crazy outliers, then find subsequent loci with distorted transmission (i.e. more than 1 base being distorted), then save all this data for subsequent analyes. 

In [3]:
source("../../BrusselSprouts/scripts/functions.R")
crosses = readLines("~/data/trd/mapped_reads/TRD.vcf.gz.samples")
crosses = crosses[startsWith(crosses, "YJNRC") | startsWith(crosses, "Chris")]
crosses


Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




[1] "ChrisC1" "ChrisC2" "ChrisC3" "ChrisC4" "ChrisC5" "ChrisC6" "ChrisC7"
 [8] "ChrisC8" "YJNRC1"  "YJNRC11" "YJNRC12" "YJNRC14" "YJNRC15" "YJNRC16"
[15] "YJNRC17" "YJNRC18" "YJNRC19" "YJNRC2"  "YJNRC20" "YJNRC21" "YJNRC22"
[22] "YJNRC23" "YJNRC24" "YJNRC25" "YJNRC26" "YJNRC27" "YJNRC29" "YJNRC3" 
[29] "YJNRC30" "YJNRC31" "YJNRC32" "YJNRC34" "YJNRC4"  "YJNRC5"  "YJNRC6" 
[36] "YJNRC7"  "YJNRC8"  "YJNRC9"

In [4]:
library(naturalsort)

In [5]:
crosses_xlsx=readxl::read_xlsx("~/data/trd/Crosses.xlsx", sheet=2)
cc=data.frame("Cross ID"=c(paste0("ChrisC",1:8)),
                                  "Short name 1"=c("ACP","BAP","CCD","ATE","ACK","AKE","BAH","ANG"),
                                  "Short name 2"=c("BFP","CMP","CPG","SACE_YCR","ACV","BAH","CGD","CEI"), stringsAsFactors=FALSE)
colnames(cc)=str_replace_all(colnames(cc), fixed("."), " ")
crosses_xlsx <- bind_rows(crosses_xlsx[,c("Cross ID","Short name 1","Short name 2")],
                        cc)
head(crosses_xlsx)
tail(crosses_xlsx)

New names:
* `Is in Stock` -> `Is in Stock...4`
* `Is in Stock` -> `Is in Stock...6`


Cross ID,Short name 1,Short name 2
<chr>,<chr>,<chr>
YJNRC1,BAK,BET
YJNRC2,BAN,BTI
YJNRC3,AKH,BQC
YJNRC4,ABS,BEF
YJNRC5,BAK,BMK
YJNRC6,BAN,CRB


Cross ID,Short name 1,Short name 2
<chr>,<chr>,<chr>
ChrisC3,CCD,CPG
ChrisC4,ATE,SACE_YCR
ChrisC5,ACK,ACV
ChrisC6,AKE,BAH
ChrisC7,BAH,CGD
ChrisC8,ANG,CEI


In [6]:
getAD <- function(x) {
  allele_count <- str_count(OH_vs_cross$ADcross[x], ",")
  AD3_0 <- as.numeric(strsplit(OH_vs_cross$ADcross[x], ",", fixed = TRUE)[[1]][1])
  AD3_1 <- as.numeric(strsplit(OH_vs_cross$ADcross[x], ",", fixed = TRUE)[[1]][2])
  if (allele_count > 1) {
    AD3_2 <- as.numeric(strsplit(OH_vs_cross$ADcross[x], ",", fixed = TRUE)[[1]][3])
    if (allele_count > 2) {
      AD3_3 <- as.numeric(strsplit(OH_vs_cross$ADcross[x], ",", fixed = TRUE)[[1]][4])
    } else {
      AD3_3 <- NA
    }
  } else {
    AD3_2 <- NA
    AD3_3 <- NA
  }
  return(data.frame(AD3_0 = AD3_0, AD3_1 = AD3_1, AD3_2 = AD3_2, AD3_3 = AD3_3))
}

isSNP <- function(x) {
  Alleles <- OH_vs_cross$alleles[x]
  Alleles <- strsplit(Alleles, ",", fixed = TRUE)[[1]]
  return(sum(str_length(Alleles) == 1) == length(Alleles))
}
PosMinus1Except1 <- function(x) {
  if (1 %in% x) {
    x[x == 1] <- x[x == 1] + 1
  }
  return(x - 1)
}

In [8]:
crosses

[1] "ChrisC1" "ChrisC2" "ChrisC3" "ChrisC4" "ChrisC5" "ChrisC6" "ChrisC7"
 [8] "ChrisC8" "YJNRC1"  "YJNRC11" "YJNRC12" "YJNRC14" "YJNRC15" "YJNRC16"
[15] "YJNRC17" "YJNRC18" "YJNRC19" "YJNRC2"  "YJNRC20" "YJNRC21" "YJNRC22"
[22] "YJNRC23" "YJNRC24" "YJNRC25" "YJNRC26" "YJNRC27" "YJNRC29" "YJNRC3" 
[29] "YJNRC30" "YJNRC31" "YJNRC32" "YJNRC34" "YJNRC4"  "YJNRC5"  "YJNRC6" 
[36] "YJNRC7"  "YJNRC8"  "YJNRC9"

In [104]:
initial_timedate=Sys.time()
jobname="getTRD"
scripts_dir="/home/jnrunge/data/trd/mapped_reads/scripts/"

for (s in 1:length(crosses)) {
    cmd=paste0("Rscript ~/TRD/02_TRD/01_getTRDs-GATK.R ",s)
    sbatch_list=execute_complex_sbatch(cmd, jobname = jobname, scripts_dir = scripts_dir, uniqueRunID = crosses[s], cores="1", mem="8G", time="short", env="JupyteR4", initial_timedate = initial_timedate, jobs_simul = 15, jobs_total = 30)

}

if(exists("sbatch_list") &  jobname == "getTRD"){
    print(sbatch_list)
    start_sbatch_list(sbatch_list, 15, jobname, initial_timedate)
}
    

[1] "/home/jnrunge/data/trd/mapped_reads/scripts/getTRD-ChrisC1.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/getTRD-ChrisC2.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/getTRD-ChrisC3.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/getTRD-ChrisC4.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/getTRD-ChrisC5.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/getTRD-ChrisC6.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/getTRD-ChrisC7.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/getTRD-ChrisC8.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/getTRD-YJNRC1.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/getTRD-YJNRC11.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/getTRD-YJNRC12.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/getTRD-YJNRC14.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/getTRD-YJNRC15.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/getTRD-YJNRC16.s